In [1]:
from mbientlab.warble import *
from mbientlab.metawear import *
from threading import Event
import time

In [2]:
e = Event()
address = None
def device_discover_task(result):
    global address
    if (result.has_service_uuid(MetaWear.GATT_SERVICE)):
        # grab the first discovered metawear device
        address = result.mac
        e.set()

BleScanner.set_handler(device_discover_task)
BleScanner.start()
e.wait()

BleScanner.stop()

In [3]:
device = MetaWear(address)
device.connect()

In [ ]:
d = device
e = Event()
# Get the accelerometer data signal
signal = libmetawear.mbl_mw_acc_get_acceleration_data_signal(d.board)
# Create a logger
logger = create_voidp(lambda fn: libmetawear.mbl_mw_datasignal_log(signal, None, fn), resource = "acc_logger")

# Start logger
libmetawear.mbl_mw_logging_start(d.board, 0)
# Turn on the accelerometer
libmetawear.mbl_mw_acc_enable_acceleration_sampling(d.board)
libmetawear.mbl_mw_acc_start(d.board)

print("Logging data for 5s")
time.sleep(5.0)

# Turn off the accelerometer
libmetawear.mbl_mw_acc_stop(d.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(d.board)
# Stop logging
libmetawear.mbl_mw_logging_stop(d.board)

print("Downloading data")
# Callback function to handle logger entries as we download them
def progress_update_handler(context, entries_left, total_entries):
    if (entries_left == 0):
        e.set()

# Function pointer and handlers for the logger download (LogDownloadHandler -> in the cbindings)
fn_wrapper = FnVoid_VoidP_UInt_UInt(progress_update_handler)
download_handler = LogDownloadHandler(context = None, received_progress_update = fn_wrapper, received_unknown_entry = cast(None, FnVoid_VoidP_UByte_Long_UByteP_UByte), received_unhandled_entry = cast(None, FnVoid_VoidP_DataP))
callback = FnVoid_VoidP_DataP(lambda ctx, p: print("{epoch: %d, value: %s}" % (p.contents.epoch, parse_value(p))))

# Stop logger
libmetawear.mbl_mw_logger_subscribe(logger, None, callback)

# Download logger contents
libmetawear.mbl_mw_logging_download(d.board, 0, byref(download_handler))
e.wait()

Exception ignored on calling ctypes callback function: <function create_voidp.<locals>.handler at 0x0000023A8D9DC9A0>
Traceback (most recent call last):
  File "C:\Users\jelte\AppData\Local\Programs\Python\Python311\Lib\site-packages\mbientlab\metawear\__init__.py", line 75, in handler
    result[0] = RuntimeError("Could not create " + (kwargs['resource'] if 'resource' in kwarg else "resource") ) if pointer == None else pointer
                                                                                        ^^^^^
NameError: name 'kwarg' is not defined


In [ ]:
device.on_disconnect = lambda status: print "we are disconnected!"
device.disconnect()